# Translates all txt files and insert result into csv


In [ ]:
import os
import time
import pandas as pd
import googletrans
from googletrans import Translator

translator = Translator()
translator.raise_Exception = True

# the text may be too long, so we will break it into chunks

def get_chunks(s, maxlength, separator):
    start = 0
    end = 0
    while start + maxlength  < len(s) and end != -1:
        end = s.rfind(separator, start, start + maxlength + 1)
        yield s[start:end]
        start = end +1
    yield s[start:]

output = ""

def translate_doc(filename):
    global df
    path = 'data/' + filename
    print(path)
    with open(path, 'r', encoding="utf-8") as f:
        lines = f.readlines()

        # count of characters
        number_of_characters = 0
        # new list to get rid of \n
        remove_n = []

        for line in lines:
                number_of_characters = number_of_characters + len(line)
                remove_n.append(line.strip())

        joined_lines = ''.join(remove_n)
        global output

        
        if number_of_characters > 5000:
            
            for group in get_chunks(joined_lines, 5000, "."):
                if len(group) < 5000:
                    print(". option", len(group))
                    group = translator.translate(group)
                    #Make list with line lengths:
                    output = output + group.text
                else:
                    for group in get_chunks(joined_lines, 5000, ";"):
                        
                        if len(group) < 5000:
                            print("; option", len(group))
                            group = translator.translate(group)
                            #Make list with line lengths:
                            output = output + group.text
                        else:
                            for group in get_chunks(joined_lines, 5000, ","):
                                # print(len(group))
                                if len(group) < 5000:
                                    print(", option", len(group))
                                    group = translator.translate(group)
                                    #Make list with line lengths:
                                    # [(n, len(n)) for n in chunks]
                                    output = output + group.text
                    
                
        else:
            print("shorter than 5000s")
            output = translator.translate(joined_lines)
            output = output.text
    return output



# iterate through the folder

def translate_all(directory):
    # create       
    data = {'File': [],
        'Translation': []}
    df = pd.DataFrame(data)

    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            translate_doc(filename)
            values_to_add = {'File': filename, 'Translation': output}
            row_to_add = pd.Series(values_to_add, name=filename)

            df = df.append(row_to_add)
            continue
        else:
            continue
    time.sleep(2)
    return df

directory = 'data/'
df = translate_all(directory)

In [ ]:
df.head(20)

In [46]:
df.to_csv('data/translated.csv', index=False)  